In [1]:
!wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/train.jsonl
!wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/val.jsonl
!wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/test.jsonl
!pip install jsonlines

--2021-01-15 16:50:15--  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2930172 (2.8M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]   2.79M  --.-KB/s    in 0.03s   

2021-01-15 16:50:16 (89.0 MB/s) - ‘train.jsonl’ saved [2930172/2930172]

--2021-01-15 16:50:16--  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/val.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 566932 (554K) [text/plain]
Saving 

In [2]:
import json
import os, re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%pylab inline
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import jsonlines
import random
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import nltk
nltk.download("all")
import re

Populating the interactive namespace from numpy and matplotlib
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading

In [3]:
sw=set(stopwords.words('russian'))

In [7]:
def cosine(text, ans):
  # tokenization 
  X_list = word_tokenize(text)  
  Y_list = word_tokenize(ans) 
  
  # sw contains the list of stopwords  
  l1 =[];l2 =[] 
    
  # remove stop words from the string 
  X_set = {w for w in X_list if not w in sw}  
  Y_set = {w for w in Y_list if not w in sw} 
    
  # form a set containing keywords of both strings  
  rvector = X_set.union(Y_set)  
  for w in rvector: 
      if w in X_set: l1.append(1) # create a vector 
      else: l1.append(0) 
      if w in Y_set: l2.append(1) 
      else: l2.append(0) 
  c = 0
    
  # cosine formula  
  for i in range(len(rvector)): 
          c+= l1[i]*l2[i] 
  cosine = c / float((sum(l1)*sum(l2))**0.5) 
  return cosine

In [8]:
def best_part(text, ques):
  senlist=re.split(r". \(\d+\)", text)
  senlist[0]=senlist[0][-(len(senlist[0])-5):]
  senlist[-1]=senlist[-1][:len(senlist[-1])-1]
  for i in range(len(senlist)):
    senlist[i]+='.'
  maxcosine=cosine(senlist[0], ques)
  ind=0
  for i in range(1, len(senlist)):
    curcosine=cosine(senlist[i], ques)
    if curcosine>maxcosine:
      maxcosine=curcosine
      ind=i
  l=max(0, ind-1)
  r=min(len(senlist)-1, ind+1)
  res=""
  for i in range(l, r+1):
    res+=senlist[i]+' '
  return res

In [9]:
def get_x_y(datajsonfile):
    x, y = [], []
    with open(datajsonfile, 'r') as jsonfile:
        jsonlist = list(jsonfile)
        for jsonstr in jsonlist:
            item = json.loads(jsonstr)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for j in questions:
                query = j['question']
                ans = j['answers']
                for i in ans:
                    #x.append([cosine(text, i['text']), cosine(best_part(text, query), i['text']), cosine(text, query), cosine(best_part(text, query+' '+i['text']), i['text'])])
                    x.append([cosine(text, i['text']), cosine(text, query), cosine(query, i['text'])])
                    y.append(i['label'])
    return x, y

In [10]:
 xtrain, ytrain=get_x_y('train.jsonl')
xtest, ytest=get_x_y('val.jsonl')
xtrain+=xtest
ytrain+=ytest

In [12]:
logreg=LogisticRegression()
logreg.fit(xtrain, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
with open('test.jsonl', 'r') as jsonfile:
  jsonlist=list(jsonfile)

In [14]:
testnew=[json.loads(i) for i in jsonlist]

In [16]:
testnew[0]['passage']['text']

'"(1) Издательство Viking Press совместно с компанией TradeMobile выпустят мобильное приложение, посвященное Анне Франк, передает The Daily Telegraph. (2) Программа будет включать в себя фрагменты из дневника Анны, озвученные британской актрисой Хеленой Бонэм Картер. (3) Помимо этого, в приложение войдут фотографии и видеозаписи, документы из архива Фонда Анны Франк, план здания в Амстердаме, где Анна с семьей скрывались от нацистов, и факсимильные копии страниц дневника. (4) Приложение, которое получит название Anne Frank App, выйдет 18 октября. (5) Интерфейс программы будет англоязычным. (6) На каких платформах будет доступно Anne Frank App, не уточняется. Анна Франк родилась в Германии в 1929 году. (7) Когда в стране начались гонения на евреев, Анна с семьей перебрались в Нидерланды. (8) С 1942 года члены семьи Франк и еще несколько человек скрывались от нацистов в потайных комнатах дома в Амстердаме, который занимала компания отца Анны. (9) В 1944 году группу по доносу обнаружили г

In [20]:
testnew[0]['passage']['questions'][0]['question']

'О чем дневник Анны Франк?'

In [17]:
def solution(anslst, question, text): #clf=clf
  anscnt=len(anslst)
  preds=[[0, i] for i in range(anscnt)]
  for i in range(anscnt):
    ans=anslst[i]['text']
    #full=best_part(text, ans)+'  &&&&&  '+question+'  |||||  '+ans
    preds[i][0]=logreg.predict_proba([[cosine(text, ans), cosine(text, question), cosine(question, ans)]])[0][1]
    anslst[i]['label']=0
  preds.sort(key=lambda x:x[0])
  preds=preds[::-1]
  onecnt=0
  p=random.randint(1, 100)
  if anscnt==2:
    onecnt=1
  elif anscnt==3:
    if p<=66:
      onecnt=1
    else:
      onecnt=2
    #onecnt=1
  elif anscnt==4:
    if p<=96:
      onecnt=2
    elif p<=98:
      onecnt=1
    else:
      onecnt=3
    #onecnt=2
  elif anscnt==5:
    if p<=97:
      onecnt=2
    elif p<=99:
      onecnt=3
    else:
      onecnt=1
    #onecnt=2
  elif anscnt==6:
    if p<=65:
      onecnt=3
    elif p<=95:
      onecnt=4
    else:
      onecnt=2
    #onecnt=3
  elif anscnt==7:
    if p<=67:
      onecnt=4
    else:
      onecnt=1
    #onecnt=4
  elif anscnt==8:
    if p<=67:
      onecnt=1
    else:
      onecnt=5
    #onecnt=1
  else:
    onecnt=4
  for i in range(onecnt):
    anslst[preds[i][1]]['label']=1
  return anslst

In [18]:
solution(testnew[0]['passage']['questions'][0]['answers'], testnew[0]['passage']['questions'][0]['question'], testnew[0]['passage']['text'])

[{'idx': 0, 'label': 0, 'text': 'Дневник об отметках Анны.'},
 {'idx': 1, 'label': 1, 'text': 'Дневник о школьных годах девочки Анны.'},
 {'idx': 2,
  'label': 0,
  'text': 'О жизни еврейской девочки во время гонения евреев нацистами.'},
 {'idx': 3,
  'label': 1,
  'text': 'Анна описывала свою жизнь и жизнь близких, когда скрывалась от нацистов.'}]

In [21]:
for i in tqdm(range(len(testnew))):
  ques=testnew[i]['passage']['questions']
  text=testnew[i]['passage']['text']
  for j in range(len(ques)):
    question=ques[j]['question']
    ques[j]['answers']=solution(ques[j]['answers'], question, text)
  testnew[i]['passage']['questions']=ques

100%|██████████| 322/322 [00:35<00:00,  9.17it/s]


In [22]:
with jsonlines.open('cosine_log_reg.jsonl', 'w') as writer:
    writer.write_all(testnew)

In [23]:
from google.colab import files
files.download('cosine_log_reg.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>